# N of the Easiest Python Synthetic Data Generators
## Let's get even more data

### Faker

https://faker.readthedocs.io/en/master/

### Scikit-learn

https://scikit-learn.org/stable/datasets/sample_generators.html

### PyOD

https://github.com/yzhao062/pyod

### CTGAN

https://github.com/sdv-dev/CTGAN

### Mimesis

https://mimesis.name/en/master/

### Data augmentation with TensorFlow

https://www.tensorflow.org/tutorials/images/data_augmentation

https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator

### 

### 

### 

### 

### 

### 

### 